<a href="https://colab.research.google.com/github/PhilanthropiCat/PiCa/blob/main/tensorflow_mn2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.datasets as datasets
import numpy as np

In [2]:
#Device
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
#Bottleneck
class Bottleneck(layers.Layer):
  def __init__(self,input_channels,output_channels,factor,stride):
    super(Bottleneck,self).__init__()
    self.conv1 = layers.Conv2D(factor*input_channels,kernel_size=(1,1))
    self.dwconv1 = layers.DepthwiseConv2D(kernel_size=(3,3),strides=stride,padding="same")
    self.conv2 = layers.Conv2D(output_channels,kernel_size=(1,1))



  def call(self,x):
    residual = x
    x = self.conv1(x)
    x = tf.nn.relu6(x)
    x = self.dwconv1(x)
    x = tf.nn.relu6(x)
    x = self.conv2(x)

    return x



In [4]:
#MobileNetV2
class MobileNetV2(keras.Model):
  def __init__(self,num_classes=10):
    super(MobileNetV2,self).__init__()
    self.resize = layers.Resizing(224,224)
    self.conv1 = keras.Sequential([
        layers.Conv2D(32,kernel_size=(3,3),strides=2,padding="same",activation="relu"),
        layers.BatchNormalization()
    ])
    self.bottleneck1 = Bottleneck(32,16,1,1)
    self.bottleneck2 = Bottleneck(16,24,6,2)
    self.bottleneck3 = Bottleneck(24,32,6,2)
    self.bottleneck4 = Bottleneck(32,64,6,2)
    self.bottleneck5 = Bottleneck(64,96,6,1)
    self.bottleneck6 = Bottleneck(96,160,6,2)
    self.bottleneck7 = Bottleneck(160,320,6,1)
    self.conv2 = keras.Sequential([
        layers.Conv2D(1280,kernel_size=(1,1),activation="relu"),
        layers.BatchNormalization()
    ])
    self.avgpool = layers.AveragePooling2D(pool_size=(7,7))
    self.flatten = layers.Flatten()
    self.dense = layers.Dense(num_classes)

  def call(self,x):
    x = self.resize(x)
    x = self.conv1(x)
    x = self.bottleneck1(x)
    x = self.bottleneck2(x)
    x = self.bottleneck3(x)
    x = self.bottleneck4(x)
    x = self.bottleneck5(x)
    x = self.bottleneck6(x)
    x = self.bottleneck7(x)
    x = self.conv2(x)
    x = self.avgpool(x)
    x = self.flatten(x)
    x = self.dense(x)
    return x

In [5]:
#Data
(x_train,y_train),(x_test,y_test) = datasets.cifar10.load_data()

In [6]:
#Compiling model
model = MobileNetV2()
model.compile(
    optimizer = keras.optimizers.Adam(0.001),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["acc"]
)

In [7]:
#Training Model
model.fit(x_train,y_train,epochs=20,batch_size=64)

Epoch 1/20
782/782 [==============================] - 106s 115ms/step - loss: 2.3036 - acc: 0.1010
Epoch 2/20
782/782 [==============================] - 89s 113ms/step - loss: 2.3031 - acc: 0.0983
Epoch 3/20
782/782 [==============================] - 88s 113ms/step - loss: 2.1241 - acc: 0.1633
Epoch 4/20
782/782 [==============================] - 88s 113ms/step - loss: 1.8637 - acc: 0.2361
Epoch 5/20
782/782 [==============================] - 88s 112ms/step - loss: 1.6443 - acc: 0.3646
Epoch 6/20
782/782 [==============================] - 88s 113ms/step - loss: 1.5173 - acc: 0.4285
Epoch 7/20
782/782 [==============================] - 88s 113ms/step - loss: 1.4285 - acc: 0.4720
Epoch 8/20
782/782 [==============================] - 88s 113ms/step - loss: 1.3484 - acc: 0.5080
Epoch 9/20
782/782 [==============================] - 88s 113ms/step - loss: 1.2645 - acc: 0.5451
Epoch 10/20
782/782 [==============================] - 88s 113ms/step - loss: 1.1974 - acc: 0.5715
Epoch 11/20
782/78

In [8]:
#Evaluating Model
loss,acc = model.evaluate(x_test,y_test)
print(f"Loss:{loss}, Accuracy:{round(acc*100)}%")

313/313 [==============================] - 7s 21ms/step - loss: 1.5959 - acc: 0.5121
Loss:1.5959330797195435, Accuracy:51%
